In [1]:
#se importan las librerias
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib qt

nm=1e-9
um=1e-6
mm=1e-3

In [ ]:
#Numero de muestras
N_x=500

#rango en el cual se van a tomar las muestras de la función
a=0
b=2*np.pi
L_x=b-a
#la distancia entre samples es:
delta_x=L_x/N_x

#defino el vector con N entradas igualmente espaciadas, avanzando como n*delta_x donde n es un entero
x=np.linspace(a,b,N_x)

#defino la función objetivo
Funcion=np.sin(50*x)+np.sin(100*x)

#defino el ancho B_x si la señal está bien muestreada:
B_x=N_x/L_x

#defino el vector del espacio de frecuencias que tiene un tamaño B_x, y un paso de 1/L_x
f_x=np.arange(0,B_x,1/L_x)

#vector vacio del tamaño de mi vector de frecuencias 
FFT_Funcion=[0]*len(f_x)

#Fourier transform
for m in range(0,len(f_x)-1):
    for n in range(0,N_x-1):
        FFT_Funcion[m]+=Funcion[n]*np.exp(-1j*2*np.pi*(n*m)/N_x)
        

magnitude=np.angle(FFT_Funcion)
plt.plot(np.abs(FFT_Funcion))
plt.show()

shift_Funcion=[0]*len(f_x)
#Fourier transform
for i in range(0,500):
        shift_Funcion[i] = FFT_Funcion[i]*np.exp(1j*2*0*np.pi/500)
        


In [55]:
#Defino función DFT

def DFT_1D(Funcion,N):
    #rango en el cual se van a tomar las muestras de la función
    a=0
    b=2*np.pi
    L_x=b-a

    #defino el vector con N entradas igualmente espaciadas, avanzando como n*delta_x donde n es un entero
    #x=np.linspace(a,b,N)

    #defino el ancho B_x si la señal está bien muestreada:
    B_x=N/L_x

    #defino el vector del espacio de frecuencias que tiene un tamaño B_x, y un paso de 1/L_x
    f_x=np.arange(0,B_x,1/L_x)

    #vector vacio del tamaño de mi vector de frecuencias 
    FFT_Funcion=[0]*len(f_x)

    #Fourier transform
    for m in range(0,len(f_x)):
        for n in range(0,N-1):
            FFT_Funcion[m]+=Funcion[n]*np.exp(-1j*2*np.pi*(n*m)/N)

    return FFT_Funcion
    

In [56]:
#Función que genera una onda esferica ideal

def spherical_wave(M,N,dx,dy,z,wavelength):
    #Centros de la imagen
    Mcen = int((M/2))
    Ncen = int((N/2))
   #Arreglos de x y y
    x=np.arange(-Mcen,Mcen+1)
    y=np.arange(-Ncen,Ncen+1)
    #Malla de la matriz
    X, Y = np.meshgrid(x,y)
    #Vector de una onda
    k= 2*np.pi/wavelength

    r=np.sqrt(z**2 + (X*dx)**2 + (Y*dy)**2)

    sph=np.exp(1j*k*r)/r

    return sph

In [2]:
#Función DFT En 2 dimensiones

def DFF_2D(entrada):
    M, N = np.shape(entrada)
    #Malla de la matriz
    dft2d = np.zeros((M,N),dtype="complex")
    
    #Fourier transform
    for i in range(0,N):
        for j in range(0,M):
         for m in range(0,N):
            for n in range(0,M):
                dft2d[i,j] +=entrada[m,n]*np.exp((-1j*2*np.pi)*((i*m/N) + (j*n/M)))

    return dft2d

In [3]:
#Apertura circular
mask=np.zeros((80,80),dtype="uint8")
#cv2.line(mask,(600,700),(77,70),1,2)
#cv2.line(mask,(600,700),(600,400),0,2)
cv2.circle(mask,(40,40),30,1,-1)
plt.imshow(mask)
plt.show()


In [30]:
#Transformada de Fresnel FFT
def TrFr(entrada,z,dx,dy,W_l):
    #Contenido frecuencial de la imagen (transformada de fourier)
    k= 2*np.pi/W_l
    N,M= np.shape(entrada)
    x=np.arange(-int(N/2),int(N/2),1)
    y=np.arange(-int(M/2),int(M/2),1)
    X, Y=np.meshgrid(x,y)
    X_0, Y_0 =np.meshgrid(x,y)

    #concidion de buen muestreo
    dx_0=(W_l*z)/(M*dx)
    dy_0=(W_l*z)/(N*dy)

    #Constante de U_0
    C1= ((1j*k)/(2*z))

    U_1 = entrada*np.exp(C1*(((X_0*dx_0)**2) + ((Y_0*dy_0)**2)))

    U_2 = np.fft.fft2(U_1)

    #Constantes de U_2
    C2= np.exp(1j*k*z)/(1j*W_l*z)
    C3= ((1j*k)/(2*z))

    U_3= U_2*C2*np.exp(C3*(((X*dx)**2) + ((Y*dy)**2)))
    
    return U_3

plt.imshow(np.abs(np.fft.fftshift((TrFr(mask,0.13,1,1,650)))))
plt.show()
    

In [219]:
#Tranferencia de Fresnel

def T_Aprox(T_f,dx,dy,z,wavelength):
    #Contenido frecuencial de la imagen (transformada de fourier)
    T_freq=np.fft.fftshift(np.fft.fft2(T_f))
    #Se define el espacio de coordenadas según la imagen que hace de U_0
    N,M= np.shape(T_f)
    x=np.arange(-int(N/2),int(N/2))
    y=np.arange(-int(M/2),int(M/2))
    X, Y=np.meshgrid(x,y)

    #dimensiones de mundo en frecuencias
    fx=X*(1/(N*dx))
    fy=Y*(1/(M*dy))

    #vector de onda Y constantes de e^{ikz} .....
    k= 2*np.pi/wavelength
    C1=np.exp(1j*k*z)
    C2=-1j*np.pi*wavelength*z

    #Tranformada H(f_x,f_y)
    Tran=C1*np.exp(C2*((fx**2)+(fy**2)))

    #U(x,y,z)
    U_resultante=Tran*T_freq

    return U_resultante


plt.imshow(np.abs(np.fft.ifft2(T_Aprox(mask,1*nm,1*nm,1*nm,650*nm))))
plt.show()

In [29]:
#Transformada de Fresnel DFT
def TrFrD(entrada,z,dx,dy,W_l):
    #Contenido frecuencial de la imagen (transformada de fourier)
    k= 2*np.pi/W_l
    N,M= np.shape(entrada)
    x=np.arange(-int(N/2),int(N/2),1)
    y=np.arange(-int(M/2),int(M/2),1)
    X, Y=np.meshgrid(x,y)
    X_0, Y_0 =np.meshgrid(x,y)

    #concidion de buen muestreo
    dx_0=(W_l*z)/(M*dx)
    dy_0=(W_l*z)/(N*dy)

    #Constante de U_0
    C1= ((1j*k)/(2*z))

    U_1 = entrada*np.exp(C1*(((X_0*dx_0)**2) + ((Y_0*dy_0)**2)))

    U_2 = DFF_2D(U_1)

    #Constantes de U_2
    C2= np.exp(1j*k*z)/(1j*W_l*z)
    C3= ((1j*k)/(2*z))

    U_3= U_2*C2*np.exp(C3*(((X*dx)**2) + ((Y*dy)**2)))
    
    return U_3

plt.imshow(np.abs(np.fft.fftshift((TrFrD(mask,0.13,1,1,650)))))
plt.show()